<a href="https://colab.research.google.com/github/georgezefko/Recommender_System/blob/main/Olist_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import seaborn as sns
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk import sent_tokenize
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings 
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
#path ='/content/drive/MyDrive/olist_project'
#path = '/content/drive/MyDrive/AdvancedBA/Olie_data/archive' #adust this one to yours
path ='/content/drive/MyDrive/DTU/ABA'

customers = pd.read_csv(path +'/olist_customers_dataset.csv')
reviews = pd.read_csv(path+'/olist_order_reviews_dataset.csv')
geolocation = pd.read_csv(path+'/olist_geolocation_dataset.csv')
items = pd.read_csv(path + '/olist_order_items_dataset.csv')
payments  = pd.read_csv(path + '/olist_order_payments_dataset.csv')
orders= pd.read_csv(path + '/olist_orders_dataset.csv')
products = pd.read_csv(path + '/olist_products_dataset.csv')
sellers =pd.read_csv(path + '/olist_sellers_dataset.csv')
category =  pd.read_csv(path + '/product_category_name_translation.csv')


In [4]:
#merge datasets

data = pd.merge(orders,customers,on='customer_id')
data = pd.merge(data,reviews,on='order_id')
data = pd.merge(data,payments,on='order_id')
data = pd.merge(data,items,on='order_id')
data = pd.merge(data,products,on='product_id')
data = pd.merge(data,sellers,on='seller_id')
data = pd.merge(data,geolocation,left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix')
data = pd.merge(data, category,on ='product_category_name')
data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1,credit_card,1,18.12,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,9350,-23.680114,-46.452454,maua,SP,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1,credit_card,1,18.12,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,9350,-23.675223,-46.441038,maua,SP,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1,credit_card,1,18.12,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,9350,-23.685712,-46.444796,mauá,SP,housewares
3,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1,credit_card,1,18.12,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,9350,-23.679951,-46.448247,maua,SP,housewares
4,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1,credit_card,1,18.12,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,9350,-23.678662,-46.442578,maua,SP,housewares


In [93]:
#choose columns to explore relevant to recommender system

recommender = data[['review_score','price','customer_state','order_status','seller_state','order_item_id','product_category_name_english','customer_unique_id','order_purchase_timestamp','geolocation_lat','geolocation_lng', 'product_id']]

In [6]:
recommender.head()

,review_score,price,customer_state,order_status,seller_state,order_item_id,product_category_name_english,customer_unique_id,order_purchase_timestamp,geolocation_lat,geolocation_lng
0,4,29.99,SP,delivered,SP,1,housewares,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:33,-23.680114,-46.452454
1,4,29.99,SP,delivered,SP,1,housewares,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:33,-23.675223,-46.441038
2,4,29.99,SP,delivered,SP,1,housewares,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:33,-23.685712,-46.444796
3,4,29.99,SP,delivered,SP,1,housewares,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:33,-23.679951,-46.448247
4,4,29.99,SP,delivered,SP,1,housewares,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:33,-23.678662,-46.442578


In [7]:
recommender.isnull().sum()

review_score                     0
price                            0
customer_state                   0
order_status                     0
seller_state                     0
order_item_id                    0
product_category_name_english    0
customer_unique_id               0
order_purchase_timestamp         0
geolocation_lat                  0
geolocation_lng                  0
dtype: int64

In [94]:
#The amount of data is huge and requires a lot of memory to run calculations let's keep only the mid top 20 (eg from 20-40)
#The first 20 ones is still huge amount of data
products = recommender['product_category_name_english'].value_counts().sort_values(ascending=False).reset_index().iloc[20:39,0].tolist()
products

['home_appliances',
 'luggage_accessories',
 'construction_tools_construction',
 'home_confort',
 'books_general_interest',
 'drinks',
 'food',
 'musical_instruments',
 'small_appliances',
 'home_construction',
 'furniture_living_room',
 'market_place',
 'audio',
 'air_conditioning',
 'food_drink',
 'home_appliances_2',
 'industry_commerce_and_business',
 'christmas_supplies',
 'agro_industry_and_commerce']

In [95]:
scores = recommender['review_score'].unique().tolist()

In [96]:
#reduce the dataset by keeping only the selected products
# keep only a sample of each rating for the selected products
frames = []
for i in products:

  df = recommender.loc[recommender['product_category_name_english'] == i]

  for j in sorted(scores):

    length = len(df[df['review_score']==j])//20
    reduced = df[df['review_score']==j].iloc[:length]
    frames.append(reduced)
  


In [97]:
sample = pd.concat(frames)
sample.head()

,review_score,price,customer_state,order_status,seller_state,order_item_id,product_category_name_english,customer_unique_id,order_purchase_timestamp,geolocation_lat,geolocation_lng,product_id
16295719,1,41.29,SP,canceled,SP,1,home_appliances,595e4f5543fcf93e124729bc947ca896,2018-02-17 09:59:29,-23.677741,-46.668647,f727147ccedba9da38dc2ab8b496a444
16295720,1,41.29,SP,canceled,SP,1,home_appliances,595e4f5543fcf93e124729bc947ca896,2018-02-17 09:59:29,-23.676303,-46.668801,f727147ccedba9da38dc2ab8b496a444
16295721,1,41.29,SP,canceled,SP,1,home_appliances,595e4f5543fcf93e124729bc947ca896,2018-02-17 09:59:29,-23.683332,-46.672478,f727147ccedba9da38dc2ab8b496a444
16295722,1,41.29,SP,canceled,SP,1,home_appliances,595e4f5543fcf93e124729bc947ca896,2018-02-17 09:59:29,-23.675802,-46.670181,f727147ccedba9da38dc2ab8b496a444
16295723,1,41.29,SP,canceled,SP,1,home_appliances,595e4f5543fcf93e124729bc947ca896,2018-02-17 09:59:29,-23.677500,-46.670550,f727147ccedba9da38dc2ab8b496a444


In [98]:
len(sample)

72313

Recommender System with Keras

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [100]:
df_keras = sample.loc[:,'customer_unique_id,product_id,review_score,order_purchase_timestamp'.split(',')]

In [101]:
customer_enc = LabelEncoder()
df_keras['customer'] = customer_enc.fit_transform(df_keras.customer_unique_id.values)
n_customers = df_keras.customer.nunique()
item_enc = LabelEncoder()
df_keras['item'] = item_enc.fit_transform(df_keras.product_id.values)
n_items = df_keras.item.nunique()
df_keras['rating'] = df_keras.review_score.values.astype(np.float32)
min_rating = min(df_keras.rating)
max_rating = max(df_keras.rating)
n_customers, n_items, min_rating, max_rating

(453, 158, 1.0, 5.0)

In [102]:
X = df_keras[['customer', 'item']].values
y = df_keras['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((65081, 2), (7232, 2), (65081,), (7232,))

In [103]:
n_factors = 10 #first hyperparameter to tune

X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [104]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
def RecommenderV1(n_customers, n_items, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_customers, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(user)
    u = Reshape((n_factors,))(u)
    
    item = Input(shape=(1,))
    m = Embedding(n_items, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(item)
    m = Reshape((n_factors,))(m)
    
    x = Dot(axes=1)([u, m])
    model = Model(inputs=[user, item], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [105]:
model = RecommenderV1(n_customers, n_items, n_factors)
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 1, 10)        4530        input_17[0][0]                   
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 1, 10)        1580        input_18[0][0]                   
____________________________________________________________________________________________

In [106]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5
1017/1017 [==============================] - 2s 2ms/step - loss: 14.7787 - val_loss: 1.5937
Epoch 2/5
1017/1017 [==============================] - 2s 2ms/step - loss: 0.8913 - val_loss: 0.1290
Epoch 3/5
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0818 - val_loss: 0.0304
Epoch 4/5
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0240 - val_loss: 0.0180
Epoch 5/5
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0158 - val_loss: 0.0162


In [107]:
from keras.layers import Add, Activation, Lambda
class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x


def RecommenderV2(n_customers, n_items, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_customers, n_factors)(user)
    ub = EmbeddingLayer(n_customers, 1)(user)
    
    item = Input(shape=(1,))
    m = EmbeddingLayer(n_items, n_factors)(item)
    mb = EmbeddingLayer(n_items, 1)(item)
    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, item], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [108]:
model = RecommenderV2(n_customers, n_items, n_factors, min_rating, max_rating)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_22 (Embedding)        (None, 1, 10)        4530        input_19[0][0]                   
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, 1, 10)        1580        input_20[0][0]                   
____________________________________________________________________________________________

In [109]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5
1017/1017 [==============================] - 3s 2ms/step - loss: 1.8905 - val_loss: 0.1256
Epoch 2/5
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0877 - val_loss: 0.0314
Epoch 3/5
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0275 - val_loss: 0.0196
Epoch 4/5
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0199 - val_loss: 0.0164
Epoch 5/5
1017/1017 [==============================] - 2s 2ms/step - loss: 0.0173 - val_loss: 0.0154


In [110]:
from keras.layers import Concatenate, Dense, Dropout
def RecommenderNet(n_customers, n_items, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_customers, n_factors)(user)
    
    item = Input(shape=(1,))
    m = EmbeddingLayer(n_items, n_factors)(item)
    
    x = Concatenate()([u, m])
    x = Dropout(0.05)(x)
    
    x = Dense(10, kernel_initializer='he_normal')(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, item], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
    return model

In [111]:
model = RecommenderNet(n_customers, n_items, n_factors, min_rating, max_rating)
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_26 (Embedding)        (None, 1, 10)        4530        input_21[0][0]                   
__________________________________________________________________________________________________
embedding_27 (Embedding)        (None, 1, 10)        1580        input_22[0][0]                   
___________________________________________________________________________________________

In [112]:
history = model.fit(x=X_train_array, y=y_train, batch_size=512, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5
128/128 [==============================] - 1s 4ms/step - loss: 2.7171 - accuracy: 0.1141 - val_loss: 1.2406 - val_accuracy: 0.1069
Epoch 2/5
128/128 [==============================] - 0s 3ms/step - loss: 1.2342 - accuracy: 0.1148 - val_loss: 0.2037 - val_accuracy: 0.1069
Epoch 3/5
128/128 [==============================] - 0s 3ms/step - loss: 0.5056 - accuracy: 0.1133 - val_loss: 0.0460 - val_accuracy: 0.1069
Epoch 4/5
128/128 [==============================] - 0s 3ms/step - loss: 0.2982 - accuracy: 0.1127 - val_loss: 0.0266 - val_accuracy: 0.1069
Epoch 5/5
128/128 [==============================] - 0s 3ms/step - loss: 0.2382 - accuracy: 0.1128 - val_loss: 0.0193 - val_accuracy: 0.1069


In [113]:
y_pred = model.predict(X_test_array)

In [114]:
y_pred = y_pred.astype(int)

In [115]:
y_pred[0]

array([1])

In [116]:
y_test[0]

1.0

In [117]:
from sklearn.metrics import accuracy_score

In [118]:
accuracy_score(y_test, y_pred)

0.27986725663716816

In [120]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred, normalize='all'))

[[1.06886062e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.50276549e-02 3.59513274e-03 2.76548673e-04 0.00000000e+00
  0.00000000e+00]
 [5.11615044e-03 1.27212389e-02 6.05641593e-02 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.69745575e-02 1.08821903e-01
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.90016593e-01
  0.00000000e+00]]


Implicit feedback

In [128]:
df_keras['timestamp'] =  pd.to_datetime(df_keras.order_purchase_timestamp, format='%Y-%m-%d %H:%M:%S')

In [129]:
df_keras['rank_latest'] = df_keras.groupby(['customer'])['timestamp'].rank(method='first', ascending=False)

train_ratings = df_keras[df_keras['rank_latest'] != 1]
test_ratings = df_keras[df_keras['rank_latest'] == 1]

# drop columns that we no longer need
train_ratings = train_ratings[['customer', 'item', 'rating']]
test_ratings = test_ratings[['customer', 'item', 'rating']]

In [130]:
train_ratings.loc[:, 'rating'] = 1

In [131]:
# Get a list of all movie IDs
all_product_ids = df_keras['item'].unique()

# Placeholders that will hold the training data
customers, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train_ratings['customer'], train_ratings['item']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in user_item_set:
    customers.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_product_ids) 
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_product_ids)
        customers.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

In [137]:
import torch
from torch.utils.data import Dataset

class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
    
    """

    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings.customer, ratings.item))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [134]:
!pip install pytorch-lightning

     |████████████████████████████████| 839kB 5.5MB/s 
     |████████████████████████████████| 276kB 9.6MB/s 
     |████████████████████████████████| 829kB 12.7MB/s 
     |████████████████████████████████| 184kB 18.8MB/s 
     |████████████████████████████████| 112kB 18.2MB/s 
     |████████████████████████████████| 1.3MB 12.1MB/s 
     |████████████████████████████████| 143kB 28.5MB/s 
     |████████████████████████████████| 296kB 28.5MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=0d2769ab84432a42de571edb23c7d6a619ae9d1da3f3a513476b20ab6a0e2942
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=1f54fe28e8734b37400b084b7bfdfb7d2cef7ac1a8baa00560a3a89a931c6dc2
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML 

In [138]:
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=512, num_workers=4)

In [139]:
num_users = df_keras.customer.max()+1
num_items = df_keras.item.max()+1
all_priduct_ids = df_keras.item.unique()

model = NCF(num_users, num_items, train_ratings, all_product_ids)

trainer = pl.Trainer(max_epochs=5, gpus=0, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 3.6 K 
1 | item_embedding | Embedding | 1.3 K 
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 2.1 K 
4 | output         | Linear    | 33    
---------------------------------------------
8.1 K     Trainable params
0         Non-trainable params
8.1 K     Total params
0.032     Total estimated model params size (MB)


1

In [142]:
# User-item pairs for testing
test_user_item_set = set(zip(test_ratings.customer, test_ratings.item))

# Dict of all items that are interacted with by each user
user_interacted_items = df_keras.groupby('customer')['item'].apply(list).to_dict()

hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_product_ids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 10 is 0.10
